In [8]:
# JHUNNIOR SAENZ data: 1995-1963
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [11]:
# Create a Service object with the executable path
chrome_path = r'C:\Users\jhunn\Downloads\chromedriver-win64\chromedriver.exe'
chrome_service = webdriver.chrome.service.Service(chrome_path)

# Use the Service object when creating the Chrome WebDriver
driver = webdriver.Chrome(service=chrome_service)
driver.maximize_window()

In [12]:
# List of URL endings for each subpage
url_endings = ['elecciones-generales-1995-presidencial_candidatos-y-resultados_8MCLSVVX8ug=CV',
               'segunda-vuelta-de-la-elección-presidencial-1990-presidencial_candidatos-y-resultados_E1U9NQHS0Xs=UQ',
               'elecciones-generales-1990-presidencial_candidatos-y-resultados_gsg4hnUOpdQ=gn',
               'elecciones-generales-1985-presidencial_candidatos-y-resultados_7Ssnc+xPt@s=s+',
               'elecciones-generales-1980-presidencial_candidatos-y-resultados_lChObgWsPqI=hg',
               'elecciones-generales-1963-presidencial_candidatos-y-resultados_rqThaZb@DOg=TZ']

# Create an empty list to store the DataFrames
dfs = []

# Loop through each URL ending
for url_ending in url_endings:
    # Go to the website
    driver.get(f"https://infogob.jne.gob.pe/Eleccion/FichaEleccion/{url_ending}")

    # Add a delay so the elements can be found
    time.sleep(15)

    # Get the HTML of the page and pass it to BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find the table and the input element
    table = soup.find('table', {'id': 'gridEleccionResultadosElectorales'})

    # Check if the table was found correctly
    if table:
        # Get the value of the input element
        input_element = soup.find('input', {'id': 'nombreEleccion'})
        year = input_element['value']

        # Find all rows in the table
        rows = table.find_all('tr')

        # Create a list to store the data
        data = []

        # Loop through each row
        for row in rows:
            # Find all columns in each row
            cols = row.find_all('td')

            # Get the text from each column
            cols = [col.text.strip() for col in cols]

            # Add the columns to the data array
            data.append(cols)

        # Create a list to store the extracted data
        extracted_data = []

        # Loop through each list in data
        for data_list in data:
            if len(data_list) >= 4:
                # Extract the first and fourth items and add them to extracted_data
                extracted_data.append([data_list[0], data_list[3]])

        # Convert the extracted data to a DataFrame
        temp_df = pd.DataFrame(extracted_data, columns=['Organización Política', 'Total de votos'])

        # Create a new DataFrame with the new column
        new_df = pd.DataFrame({'Elecciones': [year] * len(temp_df)})

        # Concatenate the new DataFrame with the original DataFrame
        temp_df = pd.concat([new_df, temp_df], axis=1)

        # Append the DataFrame to the list
        dfs.append(temp_df)
    else:
        print(f"No se encontró la tabla en la página: {url_ending}")

# Finally we close the browser
driver.quit()

# To export into excel sheet, we concatenate all DataFrames in the list into a single DataFrame
df = pd.concat(dfs)

# Specify the path where you want to store the Excel file
file_path = r'C:\Users\jhunn\Downloads\chromedriver-win64\group_2_ass_5_2024(2).xlsx'

# Use the 'to_excel' function to export the DataFrame
df.to_excel(file_path, index=False)

In [13]:
import pandas as pd
from openpyxl import load_workbook

In [14]:
# Specify the path where your Excel file is stored
file_path = r"C:\Users\jhunn\Downloads\chromedriver-win64\group_2_ass_5_2024(2).xlsx"

# Load the existing Excel file
book = load_workbook(file_path)

# Create a pandas Excel writer using openpyxl as the engine
writer = pd.ExcelWriter(file_path, engine='openpyxl')

# Assign the loaded workbook to the writer
writer.book = book

# Write the DataFrame to the Excel file
df.to_excel(writer, index=False)

# Save the changes in the writer object
writer.save()

# Close the writer and free up system resources
writer.close()

C:\Users\jhunn\AppData\Local\Temp\ipykernel_18272\1437420475.py:11: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
C:\Users\jhunn\AppData\Local\Temp\ipykernel_18272\1437420475.py:17: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
